In [ ]:
!rm -r cs224u

In [ ]:
! git clone --branch "2022-spring" https://github.com/cgpotts/cs224u.git

Cloning into 'cs224u'...
remote: Enumerating objects: 2328, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 2328 (delta 120), reused 163 (delta 106), pack-reused 2092
Receiving objects: 100% (2328/2328), 41.68 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (1419/1419), done.


In [ ]:
cd cs224u

/content/cs224u


In [ ]:
from colors import ColorsCorpusReader
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_color_describer import ContextualColorDescriber
from torch_color_describer import create_example_dataset

import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL
utils.fix_random_seeds()
COLORS_SRC_FILENAME = os.path.join(
    "filteredCorpus.csv")

In [ ]:
!wget  https://raw.githubusercontent.com/futurulus/coop-nets/master/behavioralAnalysis/humanOutput/filteredCorpus.csv

--2024-03-17 09:32:18--  https://raw.githubusercontent.com/futurulus/coop-nets/master/behavioralAnalysis/humanOutput/filteredCorpus.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9784616 (9.3M) [text/plain]
Saving to: ‘filteredCorpus.csv’

filteredCorpus.csv  100%[===================>]   9.33M  --.-KB/s    in 0.09s   

2024-03-17 09:32:18 (99.4 MB/s) - ‘filteredCorpus.csv’ saved [9784616/9784616]



In [ ]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=2,
    normalize_colors=True)
dev_examples = list(dev_corpus.read())
# This subset has about one-third the examples of the full corpus:

len(dev_examples)

13890

In [ ]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

In [ ]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

In [ ]:
def tokenize_example(s):

    # Improve me!

    return [START_SYMBOL] + s.split() + [END_SYMBOL]
tokenize_example(dev_texts_train[376])

['<s>', 'aqua,', 'teal', '</s>']

In [ ]:
def test_tokenize_example(func):
    s = "A test string"
    result = func(s)
    assert all(isinstance(tok, str) for tok in result), \
        "The tokenizer must return a list of strings."
    assert result[0] == START_SYMBOL, \
        "The tokenizer must add START_SYMBOL as the first token."
    assert result[-1] == END_SYMBOL, \
        "The tokenizer must add END_SYMBOL as the final token."

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_tokenize_example(tokenize_example)

In [ ]:
dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]

In [ ]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks})

dev_vocab += [UNK_SYMBOL]

In [ ]:
len(dev_vocab)

1439

In [ ]:
def represent_color_context(colors):

    # Improve me!

    return [represent_color(color) for color in colors]


def represent_color(color):

    # Improve me!

    return color

In [ ]:
represent_color_context(dev_rawcols_train[0])

[[0.19444444444444445, 0.5, 0.11],
 [0.7472222222222222, 0.5, 0.27],
 [0.2722222222222222, 0.5, 0.73]]

In [ ]:
def test_represent_color_context(func):
    """`func` should be `represent_color_context`"""
    example = [
        [0.786, 0.58, 0.87],
        [0.689, 0.44, 0.92],
        [0.628, 0.32, 0.81]]
    result = func(example)
    assert len(result) == len(example), \
        ("Color context representations need to represent each color "
         "separately. (We assume the final color is the target.)")
    for i, color in enumerate(result):
        assert all(isinstance(x, float) for x in color), \
            ("All color representations should be lists of floats. "
             "Color {} is {}".format(i, color))
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_represent_color_context(represent_color_context)

In [ ]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

In [ ]:
dev_mod = ContextualColorDescriber(
    dev_vocab,
    early_stopping=True)
if 'IS_GRADESCOPE_ENV' not in os.environ:
    %time _ = dev_mod.fit(dev_cols_train, dev_seqs_train)
else:
    dev_mod.fit(dev_cols_train, dev_seqs_train)

KeyboardInterrupt: 

In [ ]:
evaluation = dev_mod.evaluate(dev_cols_test, dev_seqs_test)
evaluation.keys()
evaluation['listener_accuracy']
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)
evaluation['corpus_bleu']
bleu, predicted_utterances = dev_mod.corpus_bleu(dev_cols_test, dev_seqs_test)

bleu
evaluation['target_index'][: 5]
evaluation['predicted_index'][: 5]
evaluation['predicted_utterance'][: 5]

/content/cs224u/torch_color_describer.py:91: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  color_seqs = torch.FloatTensor(color_seqs)


[['<s>',
  'mustard',
  'tan',
  'puke',
  'Middle',
  'yep!',
  'ya!',
  'thinkg',
  'grreen',
  'Box',
  'different',
  'perfect!',
  'brigther',
  'mustard',
  'muted',
  'seal',
  'those',
  'rude',
  'purple,',
  'oh'],
 ['<s>',
  'mustard',
  'tan',
  'puke',
  'Middle',
  'yep!',
  'ya!',
  'thinkg',
  'grreen',
  'Box',
  'different',
  'perfect!',
  'brigther',
  'mustard',
  'muted',
  'seal',
  'those',
  'rude',
  'purple,',
  'oh'],
 ['<s>',
  'mustard',
  'tan',
  'puke',
  'Middle',
  'yep!',
  'ya!',
  'thinkg',
  'grreen',
  'Box',
  'different',
  'perfect!',
  'brigther',
  'mustard',
  'muted',
  'seal',
  'those',
  'rude',
  'purple,',
  'oh'],
 ['<s>',
  'mustard',
  'tan',
  'puke',
  'Middle',
  'yep!',
  'ya!',
  'thinkg',
  'grreen',
  'Box',
  'different',
  'perfect!',
  'brigther',
  'mustard',
  'muted',
  'seal',
  'those',
  'rude',
  'purple,',
  'oh'],
 ['<s>',
  'mustard',
  'tan',
  'puke',
  'Middle',
  'yep!',
  'ya!',
  'thinkg',
  'grreen',
  'B

In [ ]:
dev_mod.predict(dev_cols_test[: 1])
dev_seqs_test[: 1]

[['<s>', 'right', 'side', '###', 'purple', 'pinkish', '</s>']]

In [ ]:
GLOVE_HOME = os.path.join('data', 'glove.6B')
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    pass
    # Use `utils.glove2dict` to read in the GloVe file:

    ##### YOUR CODE HERE


    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN,
    # END_TOKEN, and UNK_TOKEN are included in the embedding.

    ##### YOUR CODE HERE


    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.

    ##### YOUR CODE HERE
def test_create_glove_embedding(func):
    vocab = ['NLU', 'is', 'the', 'future', '.', '$UNK', '<s>', '</s>']
    glove_embedding, glove_vocab = func(vocab, 'glove.6B.50d.txt')
    assert isinstance(glove_embedding, np.ndarray), \
        "Expected embedding type {}; got {}".format(
        glove_embedding.__class__.__name__, glove_embedding.__class__.__name__)
    assert glove_embedding.shape == (8, 50), \
        "Expected embedding shape (8, 50); got {}".format(glove_embedding.shape)
    assert glove_vocab == vocab, \
        "Expected vocab {}; got {}".format(vocab, glove_vocab)
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_create_glove_embedding(create_glove_embedding)

In [ ]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)
len(dev_vocab)
len(dev_glove_vocab)
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab,
    embedding=dev_glove_embedding,
    early_stopping=True)
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

In [ ]:
from torch_color_describer import Decoder
import torch
import torch.nn as nn


class ColorContextDecoder(Decoder):
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)

        # Fix the `self.rnn` attribute:
        self.rnn = nn.GRU(
            input_size=self.embed_dim+self.color_dim,
            hidden_size=self.hidden_dim,
            batch_first=True)


    def get_embeddings(self, word_seqs, target_colors=None):
        """
        You can assume that `target_colors` is a tensor of shape
        (m, n), where m is the length of the batch (same as
        `word_seqs.shape[0]`) and n is the dimensionality of the
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.

        """
        ##### YOUR CODE HERE
        ret=torch.zeros((word_seqs.shape[0],word_seqs.shape[1],self.embed_dim+self.color_dim))
        for i in range(word_seqs.shape[0]):
          for j in range(word_seqs.shape[1]):
            # print(self.embedding(torch.tensor(word_seqs[i,j])))
            # print(target_colors[i])
            ret[i,j]=torch.cat([self.embedding(torch.tensor(word_seqs[i,j])),target_colors[i]])
        return ret

In [ ]:
def test_get_embeddings(decoder_class):
    """
    It's assumed that the input to this will be `ColorContextDecoder`.
    You pass in the class, and the function initalizes it with the test
    parameters.
    """
    dec = decoder_class(
        color_dim=3,   # For these, we mainly want *different*
        vocab_size=10, # dimensions so that we reliably get
        embed_dim=4,   # dimensionality errors if something
        hidden_dim=5)  # isn't working.

    # This step just changes the embedding to one with values
    # that are easy to inspect and definitely will not change
    # between runs:
    dec.embedding = nn.Embedding.from_pretrained(
        torch.FloatTensor([
            [10, 11, 12, 13],
            [14, 15, 16, 17],
            [18, 19, 20, 21]]))

    # These are the incoming sequences -- lists of indices
    # into the rows of `dec.embedding`:
    word_seqs = torch.tensor([
        [0,1,2],
        [2,0,1]])

    # Target colors as small floats that will be easy to track:
    target_colors = torch.tensor([
        [0.1, 0.2, 0.3],
        [0.7, 0.8, 0.9]])

    # The desired return value: one list of tensors for each of
    # the two sequences in `word_seqs`. Each index is replaced
    # with its vector from `dec.embedding` and has the
    # corrresponding color from `target_colors` appended to it.
    expected = torch.tensor([
        [[10., 11., 12., 13.,  0.1,  0.2,  0.3],
         [14., 15., 16., 17.,  0.1,  0.2,  0.3],
         [18., 19., 20., 21.,  0.1,  0.2,  0.3]],

        [[18., 19., 20., 21.,  0.7,  0.8,  0.9],
         [10., 11., 12., 13.,  0.7,  0.8,  0.9],
         [14., 15., 16., 17.,  0.7,  0.8,  0.9]]])

    result = dec.get_embeddings(word_seqs, target_colors=target_colors)

    assert expected.shape == result.shape, \
        "Expected shape {}; got shape {}".format(expected.shape, result.shape)

    assert torch.all(expected.eq(result)), \
        ("Your result has the desired shape but the values aren't correct. "
         "Here's what your function creates; compare it with `expected` "
         "from the test:\n{}".format(result))

In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_embeddings(ColorContextDecoder)

<ipython-input-67-37e5daaff26c>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ret[i,j]=torch.cat([self.embedding(torch.tensor(word_seqs[i,j])),target_colors[i]])


In [ ]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):

    def forward(self,
            color_seqs,
            word_seqs,
            seq_lengths=None,
            hidden=None,
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword.

        ##### YOUR CODE HERE
        # print(color_seqs[:,-1])
        # print(word_seqs)
        output,hidden = self.decoder(word_seqs,target_colors=color_seqs[:,-1],seq_lengths=seq_lengths,hidden=hidden)


        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden

In [ ]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):

    def build_graph(self):

        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:

        ##### YOUR CODE HERE
        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            hidden_dim=self.hidden_dim)


        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        return ColorizedEncoderDecoder(encoder,decoder)
        ##### YOUR CODE HERE

In [ ]:
def test_full_system(describer_class):
    toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
        group_size=50, vec_dim=2)

    toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
        train_test_split(toy_color_seqs, toy_word_seqs)

    toy_mod = describer_class(toy_vocab)

    _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

    acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

    return acc
test_full_system(ColorizedInputDescriber)

<ipython-input-72-16cd3293f7fa>:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ret[i,j]=torch.cat([self.embedding(torch.tensor(word_seqs[i,j])),target_colors[i]])
Finished epoch 1000 of 1000; error is 0.11100998520851135

1.0

In [ ]:
def evaluate_original_system(trained_model, color_seqs_test, texts_test):
    """
    Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.

    """
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:
    tok_seqs = [tokenize_example(s) for s in texts_test]

    col_seqs = [represent_color_context(colors)
                for colors in color_seqs_test]


    # Optionally include other preprocessing steps here. Note:
    # DO NOT RETRAIN YOUR MODEL AS PART OF THIS EVALUATION!
    # It's a tempting step, but it's a mistake and will get
    # you disqualified!

    # The following core score calculations are required:
    evaluation = trained_model.evaluate(col_seqs, tok_seqs)

    return evaluation

In [ ]:
my_evaluation = evaluate_original_system(dev_mod, dev_rawcols_test, dev_texts_test)
my_evaluation['listener_accuracy']
# my_evaluation['corpus_bleu']

0.33170169881946443